#Libraries

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, roc_auc_score, confusion_matrix
import xgboost as xgb
from collections import Counter
import numpy as np
import itertools
from pathlib import Path
import pandas as pd
from collections import Counter
from itertools import product
import numpy as np, pandas as pd, re
from sklearn.metrics import f1_score, recall_score, precision_score
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.metrics import classification_report
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import matplotlib.pyplot as plt
import seaborn as sns

# Loading Data-Sets

In [ ]:
def load_fasta_file(path: Path):
    seqs, labs = [], []
    lines = path.read_text().splitlines()
    for i in range(0, len(lines), 2):
        header = lines[i].strip()
        if not header:
            continue
        seq  = lines[i+1].strip() if i+1 < len(lines) else ""
        if "Positive" in header or "POSITIVE" in header or header==">1":
            labs.append(1)
        elif "Negative" in header or "NEGATIVE" in header or header==">0":
            labs.append(0)
        else:
            labs.append(0)
        seqs.append(seq.upper())
    return seqs, labs

def load_fasta_data(data_dir, files):
    seqs, labs, per_file_counts = [], [], {}
    for fname in files:
        seq, lab = load_fasta_file(Path(data_dir) / fname)
        seqs.extend(seq)
        labs.extend(lab)
        per_file_counts[fname] = Counter(lab)

    lengths = [len(s) for s in seqs]
    min_len = min(lengths)
    max_len = max(lengths)

    df = pd.DataFrame({"seq": seqs, "label": labs})

    print(f"Shortest sequence length in original dataset : {min_len}")
    print(f"Longest  sequence length in original dataset : {max_len}")

    print(f"\nTotal sequences : {len(df):,}")
    print(f"Class balance   : {Counter(df.label)}")

    print("\nBreak-down per file:")
    for f, c in per_file_counts.items():
        print(f"  {f:<12}  Pos={c[1]:>4}  Neg={c[0]:>4}")

    print("\nDataFrame preview:")
    display(df.sample(5))
    print("\n\n")
    return df

# Load data for each species using the new function
data_dir_fruit_fly = "/content/drive/MyDrive/Tri-Ori/training"
files_fruit_fly = ["bmark-BG3_F_F.txt", "bmark-KC_F_F.txt", "bmark-S2_F_F.txt"]
df_fruit_fly = load_fasta_data(data_dir_fruit_fly, files_fruit_fly)

data_dir_human = "/content/drive/MyDrive/Tri-Ori/training"
files_human = ["bmark-K562_Human.txt", "bmark-MCF7_Human.txt"]
df_human = load_fasta_data(data_dir_human, files_human)

data_dir_mouse = "/content/drive/MyDrive/Tri-Ori/training"
files_mouse = ["bmark-ES_Mouse.txt", "bmark-P19_Mouse.txt", "bmark-MEF_Mouse.txt"]
df_mouse = load_fasta_data(data_dir_mouse, files_mouse)

Shortest sequence length in original dataset : 300
Longest  sequence length in original dataset : 300

Total sequences : 18,250
Class balance   : Counter({1: 9135, 0: 9115})

Break-down per file:
  bmark-BG3_F_F.txt  Pos=2763  Neg=2764
  bmark-KC_F_F.txt  Pos=3022  Neg=3000
  bmark-S2_F_F.txt  Pos=3350  Neg=3351

DataFrame preview:


,seq,label
18129,AATGTATTATTGCCCAATGAATTGCTGCTAACTCTTGTTCTGTTGT...,0
3453,CGAAATGGGAATCAACCAAAGCCAAAGCCAAAACCAAAACCAACCC...,0
8633,CGGGCATCGCTATCCCTATCCAAATGCCAACATCTCCACATATGAA...,0
18154,AGCAACTGTCTCTGTGGTGGCTGCTGTTTCAGATGTGCTCTGTCCT...,0
11200,GTTTCAATAACTCTTTGCGGGAAATGACGAAGAGAGCTTCTCACCA...,0





Shortest sequence length in original dataset : 300
Longest  sequence length in original dataset : 300

Total sequences : 6,189
Class balance   : Counter({1: 3095, 0: 3094})

Break-down per file:
  bmark-K562_Human.txt  Pos=1332  Neg=1331
  bmark-MCF7_Human.txt  Pos=1763  Neg=1763

DataFrame preview:


,seq,label
360,TCAAGTTGGAGAGTCTCGGATTCAAGGAAGTAAACACTTTGTCCCT...,1
863,TGTGGCACGTTCATAATTCACCATCGCCTGGGATTTTGGCTCGGAA...,1
1397,CCCTGCCAGCCCCCGGAAAGTGCAGGAAGCGAAGAATGCCTGTTGG...,0
249,AGTCCCACCTCCGAGACTTCTCCCATACAGTCCCCTCCCACCCCTT...,1
2571,CTGACTCCAAATCCAGAGTTTTTACCAAAGCGACACACAGTCTTCA...,0





Shortest sequence length in original dataset : 300
Longest  sequence length in original dataset : 300

Total sequences : 8,614
Class balance   : Counter({1: 4307, 0: 4307})

Break-down per file:
  bmark-ES_Mouse.txt  Pos=1380  Neg=1380
  bmark-P19_Mouse.txt  Pos=1725  Neg=1725
  bmark-MEF_Mouse.txt  Pos=1202  Neg=1202

DataFrame preview:


,seq,label
432,GGTGCCACGCGGCTGCCGTGTGAGCAGCCGGGGGAAGCTCACCTCG...,1
7271,GAGAGGCACAGGTGGTATGCTCCACCCAGGCCCCACAGCCACTGTG...,1
8471,CTGAATTTAAAAGGTAAAGAAAATAATCCACCATCCTTCCAGAGGA...,0
470,TGGGCAGCCGTCTATGGAGTGTGGCATCTGGCCAAACTGCTATATG...,1
4888,CCAGAACAGTCATCCTTTCTTTTGCCCAGACAGGCATCCATCATAG...,0


## One-Hot Encoding

In [ ]:
def one_hot_encode_sequence(sequence):
    mapping = {
        'A': [1, 0, 0, 0],
        'C': [0, 1, 0, 0],
        'G': [0, 0, 1, 0],
        'T': [0, 0, 0, 1],
        'N': [0, 0, 0, 0]
    }
    encoded_sequence = []
    for nucleotide in sequence:
        encoded_sequence.extend(mapping.get(nucleotide, [0, 0, 0, 0]))
    return encoded_sequence

df_fruit_fly_onehot = pd.DataFrame({
    'label': df_fruit_fly['label'],
    'seq_one_hot_encoded': df_fruit_fly['seq'].apply(one_hot_encode_sequence)
})

df_human_onehot = pd.DataFrame({
    'label': df_human['label'],
    'seq_one_hot_encoded': df_human['seq'].apply(one_hot_encode_sequence)
})

df_mouse_onehot = pd.DataFrame({
    'label': df_mouse['label'],
    'seq_one_hot_encoded': df_mouse['seq'].apply(one_hot_encode_sequence)
})


print("Fruit Fly DataFrame with one-hot encoded sequences:")
display(df_fruit_fly_onehot.head(2))

print("\nHuman DataFrame with one-hot encoded sequences:")
display(df_human_onehot.head(2))

print("\nMouse DataFrame with one-hot encoded sequences:")
display(df_mouse_onehot.head(2))

Fruit Fly DataFrame with one-hot encoded sequences:


,label,seq_one_hot_encoded
0,1,"[0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, ..."
1,1,"[0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, ..."



Human DataFrame with one-hot encoded sequences:


,label,seq_one_hot_encoded
0,1,"[0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, ..."
1,1,"[0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, ..."



Mouse DataFrame with one-hot encoded sequences:


,label,seq_one_hot_encoded
0,1,"[0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, ..."
1,1,"[0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, ..."


## K-mer Counts

In [ ]:
def count_kmers(sequence, k):
    kmer_counts = Counter()
    num_kmers = len(sequence) - k + 1
    for i in range(num_kmers):
        kmer = sequence[i:i+k]
        kmer_counts[kmer] += 1
    return kmer_counts

k_size = 5

df_fruit_fly_kmer = pd.DataFrame({
    'label': df_fruit_fly['label'],
    'kmer_counts': df_fruit_fly['seq'].apply(lambda seq: count_kmers(seq, k_size))
})

df_human_kmer = pd.DataFrame({
    'label': df_human['label'],
    'kmer_counts': df_human['seq'].apply(lambda seq: count_kmers(seq, k_size))
})

df_mouse_kmer = pd.DataFrame({
    'label': df_mouse['label'],
    'kmer_counts': df_mouse['seq'].apply(lambda seq: count_kmers(seq, k_size))
})


print(f"Fruit Fly DataFrame with {k_size}-mer counts:")
display(df_fruit_fly_kmer.head(2))

print(f"\nHuman DataFrame with {k_size}-mer counts:")
display(df_human_kmer.head(2))

print(f"\nMouse DataFrame with {k_size}-mer counts:")
display(df_mouse_kmer.head(2))

Fruit Fly DataFrame with 5-mer counts:


,label,kmer_counts
0,1,"{'GATAT': 2, 'ATATT': 6, 'TATTT': 3, 'ATTTA': ..."
1,1,"{'GATGA': 1, 'ATGAC': 1, 'TGACT': 1, 'GACTC': ..."



Human DataFrame with 5-mer counts:


,label,kmer_counts
0,1,"{'TGGAG': 3, 'GGAGG': 8, 'GAGGT': 2, 'AGGTA': ..."
1,1,"{'CAGCC': 3, 'AGCCC': 3, 'GCCCA': 3, 'CCCAG': ..."



Mouse DataFrame with 5-mer counts:


,label,kmer_counts
0,1,"{'CCCCT': 2, 'CCCTA': 1, 'CCTAA': 1, 'CTAAC': ..."
1,1,"{'GGGCA': 1, 'GGCAA': 1, 'GCAAT': 1, 'CAATC': ..."


## K-mer Frequency Vectors

In [ ]:
def calculate_kmer_frequencies(kmer_counts):
    total_kmers = sum(kmer_counts.values())
    if total_kmers == 0:
        return {}
    kmer_frequencies = {kmer: count / total_kmers for kmer, count in kmer_counts.items()}
    return kmer_frequencies

df_fruit_fly_kmerVec = pd.DataFrame({
    'label': df_fruit_fly_kmer['label'],
    'kmer_frequency_vector': df_fruit_fly_kmer['kmer_counts'].apply(calculate_kmer_frequencies)
})

df_human_kmerVec = pd.DataFrame({
    'label': df_human_kmer['label'],
    'kmer_frequency_vector': df_human_kmer['kmer_counts'].apply(calculate_kmer_frequencies)
})

df_mouse_kmerVec = pd.DataFrame({
    'label': df_mouse_kmer['label'],
    'kmer_frequency_vector': df_mouse_kmer['kmer_counts'].apply(calculate_kmer_frequencies)
})


print(f"Fruit Fly DataFrame with {k_size}-mer frequency vectors:")
display(df_fruit_fly_kmerVec.head(2))

print(f"\nHuman DataFrame with {k_size}-mer frequency vectors:")
display(df_human_kmerVec.head(2))

print(f"\nMouse DataFrame with {k_size}-mer frequency vectors:")
display(df_mouse_kmerVec.head(2))

Fruit Fly DataFrame with 5-mer frequency vectors:


,label,kmer_frequency_vector
0,1,"{'GATAT': 0.006756756756756757, 'ATATT': 0.020..."
1,1,"{'GATGA': 0.0033783783783783786, 'ATGAC': 0.00..."



Human DataFrame with 5-mer frequency vectors:


,label,kmer_frequency_vector
0,1,"{'TGGAG': 0.010135135135135136, 'GGAGG': 0.027..."
1,1,"{'CAGCC': 0.010135135135135136, 'AGCCC': 0.010..."



Mouse DataFrame with 5-mer frequency vectors:


,label,kmer_frequency_vector
0,1,"{'CCCCT': 0.006756756756756757, 'CCCTA': 0.003..."
1,1,"{'GGGCA': 0.0033783783783783786, 'GGCAA': 0.00..."


# XGBoost Implementation

## GPU Detection Function

In [ ]:
import numpy as np
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from collections import Counter
import time
import warnings
warnings.filterwarnings('ignore')

In [ ]:
def check_gpu_availability():
    try:
        import torch
        if torch.cuda.is_available():
            gpu_name = torch.cuda.get_device_name(0)
            gpu_memory = torch.cuda.get_device_properties(0).total_memory / 1024**3
            return True, gpu_name, gpu_memory
    except ImportError:
        pass

    try:
        import tensorflow as tf
        if tf.config.list_physical_devices('GPU'):
            gpu_name = tf.config.list_physical_devices('GPU')[0].name
            return True, gpu_name, None
    except ImportError:
        pass

    return False, None, None

## Feature Preparation

In [ ]:
def prepare_features_for_xgboost(dataframe, feature_type='kmer_frequency'):
    kmer_freqs = dataframe['kmer_frequency_vector'].tolist()
    y = dataframe['label'].values
    return kmer_freqs, y, None

## Independent Test Loading Function

In [ ]:
# Load and combine all test files by species
INDEPENDENT_TEST_DIR = "/content/drive/MyDrive/Tri-Ori/independent"  # Updated to local workspace path

def load_and_combine_test_files():
    combined_data = {}

    # Fruit Fly: Combine BG3, KC, S2
    fruit_fly_files = ['Ind-Bg3.txt', 'Ind-KC.txt', 'Ind-S2.txt']
    fruit_fly_seqs, fruit_fly_labels = [], []
    for filename in fruit_fly_files:
        seqs, labels = load_fasta_file(Path(INDEPENDENT_TEST_DIR) / filename)
        fruit_fly_seqs.extend(seqs)
        fruit_fly_labels.extend(labels)

    combined_data['Fruit_Fly'] = pd.DataFrame({
        'seq': fruit_fly_seqs,
        'label': fruit_fly_labels
    })

    # Human: Combine K562, MCF7
    human_files = ['Ind-K562.txt', 'Ind-MCF7.txt']
    human_seqs, human_labels = [], []
    for filename in human_files:
        seqs, labels = load_fasta_file(Path(INDEPENDENT_TEST_DIR) / filename)
        human_seqs.extend(seqs)
        human_labels.extend(labels)

    combined_data['Human'] = pd.DataFrame({
        'seq': human_seqs,
        'label': human_labels
    })

    # Mouse: Combine ES, MEF, P19
    mouse_files = ['Ind-ES.txt', 'Ind-MEF.txt', 'Ind-P19.txt']
    mouse_seqs, mouse_labels = [], []

    for filename in mouse_files:
        seqs, labels = load_fasta_file(Path(INDEPENDENT_TEST_DIR) / filename)
        mouse_seqs.extend(seqs)
        mouse_labels.extend(labels)

    combined_data['Mouse'] = pd.DataFrame({
        'seq': mouse_seqs,
        'label': mouse_labels
    })

    return combined_data


## XGBoost Training Model

In [ ]:
# Import statements at the top of the cell
from sklearn.metrics import classification_report
import time

def train_xgboost_model(X, y, species_name, feature_type='kmer_frequency', test_size=0.2, fast_mode=True):
    gpu_available, gpu_name, gpu_memory = check_gpu_availability()

    # Split data
    X_train_raw, X_test_raw, y_train, y_test = train_test_split(
        X, y, test_size=test_size, random_state=42, stratify=y
    )

    # Determine vocabulary from training data
    all_kmers = set()
    for freq_dict in X_train_raw:
        all_kmers.update(freq_dict.keys())
    all_kmers = sorted(list(all_kmers))

    # Create feature matrices using training
    def create_feature_matrix(kmer_freqs, vocabulary):
        X = np.zeros((len(kmer_freqs), len(vocabulary)))
        for i, freq_dict in enumerate(kmer_freqs):
            for j, kmer in enumerate(vocabulary):
                X[i, j] = freq_dict.get(kmer, 0.0)
        return X

    X_train = create_feature_matrix(X_train_raw, all_kmers)
    X_test = create_feature_matrix(X_test_raw, all_kmers)

    if gpu_available:
        xgb_model = xgb.XGBClassifier(
            tree_method='gpu_hist',
            gpu_id=0,
            n_estimators=1000,
            max_depth=6,
            learning_rate=0.1,
            subsample=0.8,
            colsample_bytree=0.8,
            random_state=42,
            eval_metric='logloss',
            n_jobs=-1
        )
    else:
        xgb_model = xgb.XGBClassifier(
            tree_method='hist',
            n_estimators=1000,
            max_depth=6,
            learning_rate=0.1,
            subsample=0.8,
            colsample_bytree=0.8,
            random_state=42,
            eval_metric='logloss',
            n_jobs=-1
        )

    if fast_mode:
        param_grid = {
            'max_depth': [4, 6],
            'learning_rate': [0.1, 0.2],
            'n_estimators': [500, 1000]
        }
        cv = 3
    else:
        param_grid = {
            'max_depth': [3, 4, 5, 6, 7],
            'learning_rate': [0.05, 0.1, 0.15, 0.2],
            'n_estimators': [500, 800, 1000, 1200],
            'subsample': [0.8, 0.9, 1.0],
            'colsample_bytree': [0.8, 0.9, 1.0]
        }
        cv = 5

    # Grid search with improved scoring
    grid_search = GridSearchCV(
        xgb_model,
        param_grid,
        cv=cv,
        scoring='roc_auc',
        n_jobs=-1,
        verbose=1
    )

    # Train model
    start_time = time.time()
    print("🔥 Starting XGBoost training...")
    grid_search.fit(X_train, y_train)
    training_time = time.time() - start_time

    # Get best model
    best_model = grid_search.best_estimator_
    best_params = grid_search.best_params_
    best_score = grid_search.best_score_

    print(f"✅ Training completed")
    print(f"Best parameters: {best_params}")
    print(f"Best CV score: {best_score:.4f}")

    # Make predictions with explicit threshold
    y_pred_proba = best_model.predict_proba(X_test)[:, 1]
    threshold = 0.5
    y_pred = (y_pred_proba >= threshold).astype(int)

    # Calculate basic metrics
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, zero_division=0)
    recall = recall_score(y_test, y_pred, zero_division=0)
    f1_macro = f1_score(y_test, y_pred, average='macro', zero_division=0)
    auc = roc_auc_score(y_test, y_pred_proba)

    # Print classification report
    print(f"\n📊 CLASSIFICATION REPORT (80/20 Split):")
    print(classification_report(y_test, y_pred, target_names=['Negative (0)', 'Positive (1)'], digits=4))

    # Return results with vocabulary, model, and threshold for independent testing
    results = {
        'model': best_model,
        'vocabulary': all_kmers,
        'threshold': threshold,
        'k_size': 5,
        'best_params': best_params,
        'best_score': best_score,
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1_macro,
        'auc': auc,
        'training_time': training_time,
        'fast_mode': fast_mode,
        'gpu_used': gpu_available,
        'gpu_name': gpu_name,
        'feature_type': feature_type,
        'y_test': y_test,
        'y_pred': y_pred,
        'y_pred_proba': y_pred_proba,
        'X_train_raw': X_train_raw,
        'X_test_raw': X_test_raw
    }

    return results

## Batch Training

In [ ]:
def train_all_xgboost_models_with_cross_test(feature_type='kmer_frequency', fast_mode=True, combined_test_data=None):
    gpu_available, gpu_name, gpu_memory = check_gpu_availability()

    results = {}

    # Define species data
    species_data = {
        'Fruit_Fly': df_fruit_fly_kmerVec,
        'Human': df_human_kmerVec,
        'Mouse': df_mouse_kmerVec
    }

    # PHASE 1: Normal training (80/20 split)
    print(f"\n🔬 PHASE 1: TRAINING ON 80/20 SPLIT")

    for species_name, dataframe in species_data.items():
        X, y, feature_names = prepare_features_for_xgboost(dataframe, feature_type)
        species_results = train_xgboost_model(X, y, species_name, feature_type, fast_mode=fast_mode)
        results[species_name] = species_results

        print(f"✅ {species_name} training completed!")

    # PHASE 2: Cross-Species Cross-Test
    independent_results = {}

    if combined_test_data:
        print(f"\n🔬 PHASE 2: CROSS-SPECIES CROSS-TEST")

        # Test each trained model against all test datasets
        for model_species, model_info in results.items():

            model = model_info['model']
            vocabulary = model_info['vocabulary']
            threshold = model_info['threshold']
            k_size = model_info['k_size']

            independent_results[model_species] = {}

            for test_species, test_df in combined_test_data.items():
                kmer_counts = test_df['seq'].apply(lambda seq: count_kmers(seq, k_size))
                kmer_freqs = kmer_counts.apply(calculate_kmer_frequencies)

                # Create feature matrix using same vocabulary
                def create_feature_matrix_cross(kmer_freqs, vocabulary):
                    X = np.zeros((len(kmer_freqs), len(vocabulary)))
                    for i, freq_dict in enumerate(kmer_freqs):
                        for j, kmer in enumerate(vocabulary):
                            X[i, j] = freq_dict.get(kmer, 0.0)
                    return X

                X_test = create_feature_matrix_cross(kmer_freqs, vocabulary)
                y_test = test_df['label'].values

                # Make predictions
                y_pred_proba = model.predict_proba(X_test)[:, 1]
                y_pred = (y_pred_proba >= threshold).astype(int)

                # Calculate metrics
                accuracy = accuracy_score(y_test, y_pred)
                precision = precision_score(y_test, y_pred, zero_division=0)
                recall = recall_score(y_test, y_pred, zero_division=0)
                f1 = f1_score(y_test, y_pred, average='macro', zero_division=0)
                auc = roc_auc_score(y_test, y_pred_proba)

                # Store results
                independent_results[model_species][test_species] = {
                    'accuracy': accuracy,
                    'precision': precision,
                    'recall': recall,
                    'f1': f1,
                    'auc': auc,
                    'y_true': y_test,
                    'y_pred': y_pred,
                    'y_pred_proba': y_pred_proba,
                    'test_samples': len(y_test),
                    'threshold_used': threshold,
                    'k_size_used': k_size
                }

        # Print enhanced cross-check results
        print(f"\n{'='*100}")
        print("📊 CROSS-SPECIES CROSS-CHECK RESULTS")
        print(f"{'='*100}")

        for model_species in results.keys():
            print(f"\n🔬 {model_species} Model Performance:")
            for test_species in combined_test_data.keys():
                if test_species in independent_results[model_species]:
                    r = independent_results[model_species][test_species]

                    # Print detailed classification report for each cross-species test
                    print(f"\n📊 CLASSIFICATION REPORT: {model_species} Model → {test_species} Data")
                    print(classification_report(r['y_true'], r['y_pred'],
                                              target_names=['Negative (0)', 'Positive (1)'],
                                              digits=4))
                    print(f"Test samples: {r['test_samples']}")
                    print("-" * 80)

    else:
        print("\n⚠️ No combined test data provided, skipping enhanced cross-test phase")

    return results, independent_results

combined_test_data = load_and_combine_test_files()

# Execute training with combined cross-test
training_results, independent_results = train_all_xgboost_models_with_cross_test(
    feature_type='kmer_frequency',
    fast_mode=True,
    combined_test_data=load_and_combine_test_files()
)


🔬 PHASE 1: TRAINING ON 80/20 SPLIT
🔥 Starting XGBoost training...
Fitting 3 folds for each of 8 candidates, totalling 24 fits
✅ Training completed
Best parameters: {'learning_rate': 0.1, 'max_depth': 6, 'n_estimators': 1000}
Best CV score: 0.8388

📊 CLASSIFICATION REPORT (80/20 Split):
              precision    recall  f1-score   support

Negative (0)     0.7662    0.7926    0.7792      1823
Positive (1)     0.7857    0.7586    0.7719      1827

    accuracy                         0.7756      3650
   macro avg     0.7759    0.7756    0.7756      3650
weighted avg     0.7760    0.7756    0.7756      3650

✅ Fruit_Fly training completed!
🔥 Starting XGBoost training...
Fitting 3 folds for each of 8 candidates, totalling 24 fits
✅ Training completed
Best parameters: {'learning_rate': 0.1, 'max_depth': 6, 'n_estimators': 500}
Best CV score: 0.9124

📊 CLASSIFICATION REPORT (80/20 Split):
              precision    recall  f1-score   support

Negative (0)     0.8214    0.8546    0.8377    